In [88]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, isnan, when, count, col, to_date, regexp_replace, date_format
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))

spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.memory.offHeap.size','4g')
    .config('spark.executor.memory', '16g')
    .appName("sapient")
    .getOrCreate()
)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 01:18:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

# ecarbro.json, AIA-1-25.ecar.json, conn.09_00_00-10_00_00.log
def s3_file(file):
    dev_file = f"{s3_url_raw}/{env}/{file}"
    prod_file = ""
    if env == "prod":
      filename = prod_file
    else:
      filename = dev_file
    return filename

def loadAndCheckpoint(type):
    """
    type: ecar, ecar-bro, bro, labels
    This function reads a file from json or log text and writes it as a parquet.
    """
    ecar_fil = [("FLOW"), ("PROCESS"), ("FILE"),("SHELL")]
    start_time = time.time()
    if type == 'ecar':
        #adding filter to dispense objects not included in the list above. 
        df = spark.read.json(f"{s3_url_raw}/{env}/{type}/**/**/**/*.json").filter(col("Object").isin(ecar_fil)) #.filter((col("object") == "FILE") | (col("object") == "FLOW") 
                                #| (col("object") == "PROCESS") | (col("object") == "SHELL"))
        df = df.limit(5000)
        df = df.select(*df.columns, "properties.*").drop('properties')
        df.write.option("maxRecordsPerFile", 100000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/{type}")
    elif type == 'ecar-bro':
        df = spark.read.json(f"{s3_url_raw}/{env}/{type}/**/**/**/*.json")#.filter((col("object") == "FILE") | (col("object") == "FLOW") 
                                #| (col("object") == "PROCESS") | (col("object") == "SHELL"))
        # this will extract and flatten nested properties column
        df = df.limit(1000)
        df = df.select(*df.columns, "properties.*").drop('properties')
        df.write.option("maxRecordsPerFile", 100000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/{type}")
    elif type == 'bro':
        df = spark.read.csv(f"{s3_url_raw}/{env}/**/**/*.log", sep="\t", comment="#", header=False)#.filter(col("`id.resp_p`") != 443)
        #df = df.limit(1000)
        df = df.toDF(*bro_cols_conn)
        df.write.option("maxRecordsPerFile", 100000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/{type}")
    elif type == 'labels':
        df = spark.read.csv(f"{s3_url_raw}/{env}/{type}/*.csv", sep=",", header=True)
        df.write.option("maxRecordsPerFile", 100000).mode("overwrite").parquet(f"{s3_url_refined}/{env}/{type}")
    print("--- %s seconds ---" % (time.time() - start_time))
    df.unpersist()
    
def readCheckpoint(file_type):
    """
    type: ecar, ecar-bro, bro
    """
    s3_parquet_loc = f"{s3_url_refined}/{env}/{file_type}"
    start_time = time.time()
    df = spark.read.parquet(s3_parquet_loc)
    # rdd = spark.sparkContext.parallelize(df.take(1000))
    # print(f"Your dataframe has {rdd.count():,} rows.")
    print("--- %s seconds ---" % (time.time() - start_time))
    return df

# env can be dev or prod
env = "dev"


In [5]:
loadAndCheckpoint("labels")
df_labels = readCheckpoint('labels')

23/02/21 01:19:26 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


--- 92.82696843147278 seconds ---
--- 0.8219571113586426 seconds ---


In [7]:
df_labels.printSchema()

root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)



In [8]:
df_labels.limit(5).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action
0,SysClient0010.systemia.com,56036068-0c69-49ff-9630-6c402031fc7f,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.665-04:00,SHELL,COMMAND
1,SysClient0010.systemia.com,89e62e88-a429-4e8b-a334-27c8b7276ba3,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.666-04:00,SHELL,COMMAND
2,SysClient0010.systemia.com,f2265f4a-fa24-46b2-98e3-51c19b85f446,244580f1-dd97-4afc-a249-4ed571a0cd77,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.963-04:00,FLOW,MESSAGE
3,SysClient0010.systemia.com,98f2c5b5-0a43-420e-9201-cecba96624b8,1829c1a0-a7b0-4cbb-843a-5b947a744500,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.964-04:00,FLOW,MESSAGE
4,SysClient0010.systemia.com,587fd978-0239-44a3-9780-d3fe3bbd2e72,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.75-04:00,SHELL,COMMAND


In [72]:
#take "T" out of the timestamp column because it was throwing the to_date filters off
df_labels2 = df_labels.withColumn('new_date', regexp_replace('timestamp', 'T', ''))

In [77]:
df_labels2.limit(10).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action,new_date
0,SysClient0010.systemia.com,56036068-0c69-49ff-9630-6c402031fc7f,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.665-04:00,SHELL,COMMAND,2019-09-2501:28:37.665-04:00
1,SysClient0010.systemia.com,89e62e88-a429-4e8b-a334-27c8b7276ba3,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.666-04:00,SHELL,COMMAND,2019-09-2501:28:37.666-04:00
2,SysClient0010.systemia.com,f2265f4a-fa24-46b2-98e3-51c19b85f446,244580f1-dd97-4afc-a249-4ed571a0cd77,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.963-04:00,FLOW,MESSAGE,2019-09-2501:28:37.963-04:00
3,SysClient0010.systemia.com,98f2c5b5-0a43-420e-9201-cecba96624b8,1829c1a0-a7b0-4cbb-843a-5b947a744500,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.964-04:00,FLOW,MESSAGE,2019-09-2501:28:37.964-04:00
4,SysClient0010.systemia.com,587fd978-0239-44a3-9780-d3fe3bbd2e72,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.75-04:00,SHELL,COMMAND,2019-09-2501:29:07.75-04:00
5,SysClient0010.systemia.com,00e8f1d2-d3d7-4559-9e50-1f130b7dab54,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.752-04:00,SHELL,COMMAND,2019-09-2501:29:07.752-04:00
6,SysClient0010.systemia.com,ca57c9be-fa79-471d-9899-467ca6cf6533,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.768-04:00,SHELL,COMMAND,2019-09-2501:29:07.768-04:00
7,SysClient0010.systemia.com,7dc7f25e-4e17-48a1-8333-2e67516d4d30,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.77-04:00,SHELL,COMMAND,2019-09-2501:29:07.77-04:00
8,SysClient0010.systemia.com,2ca843d5-ebdd-44cb-a1a5-c1de1d120742,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.774-04:00,SHELL,COMMAND,2019-09-2501:29:07.774-04:00
9,SysClient0010.systemia.com,28a7cfdd-b097-440c-8531-07467374ce97,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.775-04:00,SHELL,COMMAND,2019-09-2501:29:07.775-04:00


In [98]:
#pull out malicious events from first day and count those events.
#to do so I am attempting to create a date column for ordering. I am also making another df with 
#a third date column to check the granularity of the date objects. Thus far, I can only generate day level
#granularity. 
df_labels_date = df_labels2.withColumn("date",to_date("new_date", "yyyy-MM-ddHH:mm:ss.SSSXXXXX"))
df_labels_date2 = df_labels_date.withColumn("date_str",date_format("date", "yyyy-MM-dd HH:mm:ss.SSS"))

In [90]:
df_labels_date.printSchema()

root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- new_date: string (nullable = true)
 |-- date: date (nullable = true)



In [91]:
df_labels_date2.printSchema()

root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- new_date: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_str: string (nullable = true)



In [99]:
df_labels_date.limit(5).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action,new_date,date
0,SysClient0010.systemia.com,56036068-0c69-49ff-9630-6c402031fc7f,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.665-04:00,SHELL,COMMAND,2019-09-2501:28:37.665-04:00,2019-09-25
1,SysClient0010.systemia.com,89e62e88-a429-4e8b-a334-27c8b7276ba3,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.666-04:00,SHELL,COMMAND,2019-09-2501:28:37.666-04:00,2019-09-25
2,SysClient0010.systemia.com,f2265f4a-fa24-46b2-98e3-51c19b85f446,244580f1-dd97-4afc-a249-4ed571a0cd77,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.963-04:00,FLOW,MESSAGE,2019-09-2501:28:37.963-04:00,2019-09-25
3,SysClient0010.systemia.com,98f2c5b5-0a43-420e-9201-cecba96624b8,1829c1a0-a7b0-4cbb-843a-5b947a744500,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.964-04:00,FLOW,MESSAGE,2019-09-2501:28:37.964-04:00,2019-09-25
4,SysClient0010.systemia.com,587fd978-0239-44a3-9780-d3fe3bbd2e72,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.75-04:00,SHELL,COMMAND,2019-09-2501:29:07.75-04:00,2019-09-25


In [93]:
df_labels_date2.limit(5).toPandas()

,hostname,id,objectID,actorID,timestamp,object,action,new_date,date,date_str
0,SysClient0010.systemia.com,56036068-0c69-49ff-9630-6c402031fc7f,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.665-04:00,SHELL,COMMAND,2019-09-2501:28:37.665-04:00,2019-09-25,2019-09-25 00:00:00.000
1,SysClient0010.systemia.com,89e62e88-a429-4e8b-a334-27c8b7276ba3,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.666-04:00,SHELL,COMMAND,2019-09-2501:28:37.666-04:00,2019-09-25,2019-09-25 00:00:00.000
2,SysClient0010.systemia.com,f2265f4a-fa24-46b2-98e3-51c19b85f446,244580f1-dd97-4afc-a249-4ed571a0cd77,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.963-04:00,FLOW,MESSAGE,2019-09-2501:28:37.963-04:00,2019-09-25,2019-09-25 00:00:00.000
3,SysClient0010.systemia.com,98f2c5b5-0a43-420e-9201-cecba96624b8,1829c1a0-a7b0-4cbb-843a-5b947a744500,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:28:37.964-04:00,FLOW,MESSAGE,2019-09-2501:28:37.964-04:00,2019-09-25,2019-09-25 00:00:00.000
4,SysClient0010.systemia.com,587fd978-0239-44a3-9780-d3fe3bbd2e72,8b335bf1-f27e-42ff-80b9-8c60c25eb212,3a3de01b-44d0-4a13-95fb-d6c2c318ec3d,2019-09-25T01:29:07.75-04:00,SHELL,COMMAND,2019-09-2501:29:07.75-04:00,2019-09-25,2019-09-25 00:00:00.000
